In [6]:
import os
import operator
from num2words import num2words  # 这个包不支持中文
import gc
import time


In [2]:
SUB = str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789")
SUP = str.maketrans("⁰¹²³⁴⁵⁶⁷⁸⁹", "0123456789")
OTH = str.maketrans("፬", "4")

In [3]:
train = open("input/en_train.csv", encoding='UTF8')

In [4]:
line = train.readline()

In [5]:
line

'"sentence_id","token_id","class","before","after"\n'

根据训练数据，生成一个大辞典：res，并保留出现次数。

In [7]:
res = {}
total = 0
not_same = 0
s = time.time()
while True:
    line = train.readline().strip()
    if line == '':
        break
    total += 1
    pos = line.find('","')
    text = line[pos + 2:]
    if text[:3] == '","':
        continue
    text = text[1:-1]
    arr = text.split('","')
    if arr[0] != arr[1]:
        not_same += 1
    if arr[0] not in res:
        res[arr[0]] = dict()
        res[arr[0]][arr[1]] = 1
    else:
        if arr[1] in res[arr[0]]:
            res[arr[0]][arr[1]] += 1
        else:
            res[arr[0]][arr[1]] = 1
train.close()
print('train file:\tTotal: {} Have diff value: {}'.format(total, not_same))
print(time.time()-s)

train file:	Total: 9918441 Have diff value: 659793
20.64557909965515


In [7]:
res

{'': {'': 37, 'n a': 5},
 'Yangyang': {'Yangyang': 1},
 'Jewels': {'Jewels': 25},
 'Circuito': {'Circuito': 4},
 'ROCAF': {'r o c a f': 1},
 'Meloy': {'Meloy': 6},
 'Imathia': {'Imathia': 1},
 'NewMediaRockstars': {'NewMediaRockstars': 1},
 'tanha': {'tanha': 1},
 '$78,252': {'seventy eight thousand two hundred fifty two dollars': 1},
 'V.N.': {'v n': 3},
 'Pachylemur': {'Pachylemur': 2},
 'Eirlys': {'Eirlys': 1},
 'psychodynamics': {'psychodynamics': 2},
 'Lav': {'Lav': 1},
 'Spot': {'Spot': 43},
 'Plessen': {'Plessen': 1},
 'Heerema': {'Heerema': 1},
 'QXFM': {'q x f m': 1},
 'Flanders': {'Flanders': 66},
 'Kasko': {'Kasko': 1},
 'Jikeikai': {'Jikeikai': 1},
 'Aiton': {'Aiton': 4},
 'geophytes': {'geophytes': 1},
 'Simonoff': {'Simonoff': 1},
 'MRBMs': {"m r b m's": 1},
 'wonder': {'wonder': 37},
 '2011-11-04': {'the fourth of november twenty eleven': 2},
 '$90,000,000': {'ninety million dollars': 1},
 '484.2/km2': {'four hundred eighty four point two per square kilometers': 1},
 'Ga

In [8]:
total = 0
changes = 0
out = open('baseline_en.csv', "w", encoding='UTF8')
out.write('"id","after"\n')
test = open("input/en_test.csv", encoding='UTF8')
line = test.readline().strip()
while 1:
    line = test.readline().strip()
    if line == '':
        break

    pos = line.find(',')
    i1 = line[:pos]
    line = line[pos + 1:]

    pos = line.find(',')
    i2 = line[:pos]
    line = line[pos + 1:]

    line = line[1:-1]
    out.write('"' + i1 + '_' + i2 + '",')
    if line in res:
        srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
        out.write('"' + srtd[0][0] + '"')
        changes += 1
    else:
        out.write('"' + line + '"')

    out.write('\n')
    total += 1

print('Total: {} Changed: {}'.format(total, changes))
test.close()
out.close()

Total: 1088564 Changed: 999105


In [ ]:
files = os.listdir(DATA_INPUT_PATH)
for file in files:
    train = open(os.path.join(DATA_INPUT_PATH, file), encoding='UTF8')
    while 1:
        line = train.readline().strip()
        if line == '':
            break
        total += 1
        pos = line.find('\t')
        text = line[pos + 1:]
        if text[:3] == '':
            continue
        arr = text.split('\t')
        if arr[0] == '<eos>':
            continue
        if arr[1] != '<self>':
            not_same += 1

        if arr[1] == '<self>' or arr[1] == 'sil':
            arr[1] = arr[0]

        if arr[1] == '<self>' or arr[1] == 'sil':
            arr[1] = arr[0]

        if arr[0] not in res:
            res[arr[0]] = dict()
            res[arr[0]][arr[1]] = 1
        else:
            if arr[1] in res[arr[0]]:
                res[arr[0]][arr[1]] += 1
            else:
                res[arr[0]][arr[1]] = 1
    train.close()
    print(file + ':\tTotal: {} Have diff value: {}'.format(total, not_same))
    gc.collect()

sdict = {}
sdict['km2'] = 'square kilometers'
sdict['km'] = 'kilometers'
sdict['kg'] = 'kilograms'
sdict['lb'] = 'pounds'
sdict['dr'] = 'doctor'
sdict['m²'] = 'square meters'

total = 0
changes = 0
out = open(os.path.join(SUBM_PATH, 'baseline_ext_en.csv'), "w", encoding='UTF8')
out.write('"id","after"\n')
test = open(os.path.join(INPUT_PATH, "en_test.csv"), encoding='UTF8')
line = test.readline().strip()
while 1:
    line = test.readline().strip()
    if line == '':
        break

    pos = line.find(',')
    i1 = line[:pos]
    line = line[pos + 1:]

    pos = line.find(',')
    i2 = line[:pos]
    line = line[pos + 1:]

    line = line[1:-1]
    out.write('"' + i1 + '_' + i2 + '",')
    if line in res:
        srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
        out.write('"' + srtd[0][0] + '"')
        changes += 1
    else:
        if len(line) > 1:
            val = line.split(',')
            if len(val) == 2 and val[0].isdigit and val[1].isdigit:
                line = ''.join(val)

        if line.isdigit():
            srtd = line.translate(SUB)
            srtd = srtd.translate(SUP)
            srtd = srtd.translate(OTH)
            out.write('"' + num2words(float(srtd)) + '"')
            changes += 1
        elif len(line.split(' ')) > 1:
            val = line.split(' ')
            for i, v in enumerate(val):
                if v.isdigit():
                    srtd = v.translate(SUB)
                    srtd = srtd.translate(SUP)
                    srtd = srtd.translate(OTH)
                    val[i] = num2words(float(srtd))
                elif v in sdict:
                    val[i] = sdict[v]

            out.write('"' + ' '.join(val) + '"')
            changes += 1
        else:
            out.write('"' + line + '"')

    out.write('\n')
    total += 1

print('Total: {} Changed: {}'.format(total, changes))
test.close()
out.close()